In [1]:
import warnings
warnings.filterwarnings("ignore")

# Introducción
Los sistemas de recomendación son algoritmos diseñados para sugerir elementos relevantes a los usuarios. Su objetivo principal es predecir la “calificación” o “preferencia” que un usuario daría a un ítem.

Existen tres enfoques principales:
1. **Filtrado Colaborativo (CF):** se apoya en similitudes entre usuarios o ítems.
2. **Basado en Contenido:** usa características de los ítems para recomendar.
3. **Modelos Híbridos:** combinan ambos.


### 1. Filtrado Colaborativo (CF)

Este es el enfoque más común, realiza predicciones basadas en los comportamientos pasados de un gran grupo de usuarios.

Hay dos subtipos principales:

- -  **Filtrado colaborativo basado en usuarios:** encuentra usuarios similares al usuario objetivo (es decir, que han calificado ítems de manera parecida en el pasado) y recomienda ítems que estos usuarios similares han disfrutado pero que el usuario objetivo aún no ha visto.

*Analogía:* es como preguntar a tus amigos con gustos similares en películas que te recomienden una.

- - **Filtrado colaborativo basado en ítems:** en lugar de buscar usuarios similares, encuentra ítems similares a los que el usuario ya ha calificado positivamente. Calcula la similitud en función de quién los ha visto, gustado o comprado. Ejemplo: usuarios que compraron Álgebra Lineal y sus Aplicaciones también compraron con frecuencia Introducción a los Algoritmos.

 Es como la función de Amazon “Clientes que compraron este producto también compraron...”.

**Ventajas:**

- No requiere conocer las características de los ítems ni de los usuarios -(agnóstico al contenido).

- Puede generar recomendaciones inesperadas (serendipia), sugiriendo ítems que el usuario no habría descubierto por sí mismo.

**Desventajas:**

- Problema de arranque en frío: difícil recomendar a nuevos usuarios o evaluar ítems nuevos sin historial.

- Escasez de datos: el rendimiento baja cuando la matriz de interacciones usuario-ítem es muy dispersa.

#### Factorización Matricial (SVD)
Una técnica muy usada en CF es la inspirada en la **factorización matricial (Singular Value Descomposition)**:

Descompone la gran matriz dispersa de interacciones usuario-ítem (R) en dos matrices más pequeñas y densas:

- Una de características de usuarios (P)  
- Otra de características de ítems (Q)  

El producto punto entre un vector de usuario en (P) y uno de ítem en (Q) aproxima la calificación esperada:

$$
R \approx P\, Q^{\top}
$$

Con esto podemos predecir la preferencia de un usuario por un ítem.

Pero si los datos de \(R\) están sesgados (ej. solo vemos películas populares), entonces la factorización **también aprenderá el sesgo**.

## El Problema de los Confusores
- **Confusores (confounders):** Son variables que afectan tanto la exposición como el resultado.
- Ejemplos:
  - Popularidad: lo más visto parece “mejor”.
  - Posición en pantalla: lo que aparece primero recibe más clics.
  - Contexto: día, dispositivo, campañas, etc.

### Consecuencias:
- **Sesgo hacia lo popular.**
- **Feedback loop:** lo popular se recomienda más → genera más datos → se recomienda más.  
- **Evaluación sesgada:** las métricas tradicionales pueden sobreestimar el rendimiento.

Por lo tanto necesitamos técnicas que **corrijan este sesgo**.

## Ejemplo
Para abordar el problema anterior realizaremos un sistema de recomendacion con la base movielens la cual contiene información de calificaciones y etiquetas de películas recolectadas en la plataforma [MovieLens](http://movielens.org), un sistema de recomendación de películas desarrollado por **GroupLens Research** de la Universidad de Minnesota.

- **Tamaño del dataset:**  
  - **100,836** calificaciones  
  - **3,683** etiquetas (tags)  
  - **9,742** películas  
  - **610** usuarios  

- **Periodo de recolección:** Desde **29 de marzo de 1996** hasta **24 de septiembre de 2018**.  
- **Última generación del dataset:** **26 de septiembre de 2018**.  
- **Condición de selección de usuarios:** cada usuario incluido ha calificado al menos **20 películas**.  
- **Nota:** No incluye datos demográficos de los usuarios (solo un `userId` anónimo).

### Archivos del Dataset
El dataset está compuesto por cuatro archivos CSV principales:

#### 1. `ratings.csv`
Contiene todas las calificaciones hechas por los usuarios.
- **Variables:**  userId, movieId, rating, timestamp

- **Detalles:**  
 - `rating` va de **0.5 a 5.0** en incrementos de 0.5 estrellas.  
 -  `timestamp` representa segundos desde el 1 de enero de 1970 (UTC).

#### 2. `tags.csv`
Contiene las etiquetas (palabras clave o frases cortas) aplicadas por usuarios a las películas.
- **Variables:** userId, movieId, tag, timestamp

- **Detalles:**  
 - `tag` son metadatos generados por usuarios.  
 - `timestamp` en el mismo formato que en `ratings.csv`.

#### 3. `movies.csv`
Contiene la información básica de cada película.
- **Variables:** movieId, title, genres

- **Detalles:**  
 - `title`: nombre de la película con el año en paréntesis (ej. *Toy Story (1995)*).  
 - `genres`: lista separada por barras verticales `|` (ej. *Adventure|Animation|Children|Comedy|Fantasy*).  
 - Posibles géneros incluyen: *Action, Adventure, Animation, Comedy, Drama, Fantasy, Horror, Romance, Sci-Fi, Thriller, War, Western,* entre otros.

#### 4. `links.csv`
Contiene identificadores que permiten enlazar las películas con otras bases de datos.
- **Variables:**  movieId,imdbId,tmdbId
- **Detalles:**  
 - `imdbId`: corresponde al identificador de [IMDb](http://www.imdb.com).  
 - `tmdbId`: corresponde al identificador de [The Movie Database (TMDb)](https://www.themoviedb.org).  


 ### Baseline sin corrección (SVD simple)

In [2]:
import pandas as pd

# Cargamos ratings y movies
ratings = pd.read_csv( "./input/ratings.csv")
movies = pd.read_csv("./input/movies.csv")

ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.rating.value_counts()

rating
4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: count, dtype: int64

### El problema del sesgo en la factorización

La **factorización matricial (SVD)** nos permite aproximar la matriz de interacciones:

$$
R \approx P Q^T
$$

donde:
- $P$ representa características latentes de usuarios.  
- $Q$ representa características latentes de ítems.  

Con esto podemos predecir la preferencia de un usuario por un ítem.

**Pero hay un problema importante:** si los datos de $R$ están sesgados (ejemplo: solo vemos películas populares porque fueron más expuestas), entonces **la factorización también aprenderá ese sesgo**.  

En la práctica, esto significa que el modelo recomendará **más de lo mismo** (lo popular), reforzando el bucle de retroalimentación.


### Para corregir el sesgo: IPS

La clave está en reconocer que **no todos los pares usuario–ítem tienen la misma probabilidad de aparecer en los datos**.  
A esto lo llamamos **propensión**:

$$
e_{ui} = \Pr(A_{ui} = 1 \mid u,i)
$$

Donde:

- $u$: índice del **usuario** (ej. Juan, usuario #15).  
- $i$: índice del **ítem** (ej. Película X, ítem #203).
- $A_{ui}=1$ significa que el usuario $u$ tuvo la oportunidad de ver el ítem $i$.
- $e_{ui}$: la **propensión**, es decir, la probabilidad de que el par usuario–ítem $(u,i)$ se **muestre** en los datos.  
  - Ejemplo: si una película aparece en la página principal y la ven el 80% de los usuarios, su $e_{ui}$ será alto.  
  - Si una película rara apenas aparece, su $e_{ui}$ será bajo.


#### Inverse Propensity Scoring (IPS)
La idea es **reponderar cada observación** de acuerdo a su propensión.  
Así, los pares muy expuestos (ej. ítems populares) pesan menos, y los pares poco expuestos pesan más.

$$
w_{ui} = \frac{1}{e_{ui}}
$$

- Si $e_{ui}$ es **alto** (ítem popular, siempre mostrado) → $w_{ui}$ es pequeño.  
- Si $e_{ui}$ es **bajo** (ítem de nicho, poco mostrado) → $w_{ui}$ es grande.  

Esto balancea el entrenamiento y reduce el sesgo de popularidad.


### Ejemplo IPS

Imaginemos dos películas:

- **Película A (blockbuster):** fue mostrada al **90%** de los usuarios.  
  - Propensión: $e_{ui} = 0.9$
  - Peso IPS:  
  $$
  w_{ui} = \frac{1}{0.9} \approx 1.11
  $$

- **Película B (de nicho):** fue mostrada solo al **10%** de los usuarios.  
  - Propensión: $e_{ui} = 0.1$
  - Peso IPS:  
  $$
  w_{ui} = \frac{1}{0.1} = 10
  $$


### Interpretación

- Si un error ocurre con **Película A** (muy popular), su impacto en el entrenamiento es bajo $(w \approx 1)$.  
- Si un error ocurre con **Película B** (poco expuesta), su impacto es **10 veces mayor**.

Esto significa que el modelo aprende a **no dejar de lado los ítems poco expuestos**, corrigiendo el sesgo hacia lo popular.

### La corrección con IPS
En IPS, no reemplazamos \(R\), sino que ajustamos **la importancia de cada entrada** en la factorización.  

$$
\tilde{R}_{ui} = w_{ui} \cdot R_{ui} \quad \text{con} \quad w_{ui} = \frac{1}{e_{ui}}
$$

- $\tilde{R}$ = versión **reponderada** de la matriz de interacciones.  

### Factorización reponderada

En lugar de minimizar solo el error sobre $R$, resolvemos:

$$
\min_{P,Q} \sum_{(u,i)\in \Omega} w_{ui} \, \big( R_{ui} - P_u \cdot Q_i \big)^2
$$

Donde:
- $\Omega$ es el conjunto de pares usuario–ítem observados.  
- Cada error se multiplica por $w_{ui}$.  


### Implementación

En nuestro caso:
- Usaremos una **regresión logística** para la propensión.
- Calculamos los pesos $w_{ui}$ (con clipping para evitar valores extremos).  
- Entrenamos un modelo de factorización.  

En librerías como **LightFM**, esto se implementa fácilmente pasando los $w_{ui}$ como `sample_weight` en el entrenamiento.

El resultado sera un modelo que no solo busca **minimizar el error**, sino que además **corrige el sesgo de exposición**, generando recomendaciones más justas y diversas.

In [5]:
movies[movies['movieId']==356]

,movieId,title,genres
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War


### Comparación: Baseline vs IPS
Veremos:
1. RMSE (aprox. igual o un poco peor con IPS, pero más **justo**).
2. **Popularidad promedio** de ítems recomendados (con IPS podría bajar).

In [ ]:
#!pip install surprise
!pip install "numpy<2"


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [6]:
#!pip uninstall numpy -y
#!pip install "numpy<2"
import numpy as np
print(np.__version__)

1.26.4


In [2]:
from surprise import Dataset, Reader, SVD

from sklearn.metrics import mean_squared_error
from lightfm import LightFM
from scipy import sparse
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import random

ModuleNotFoundError: No module named 'surprise'

In [9]:
#  SPLIT ÚNICO 

# Última interacción de cada usuario como test
r_sorted = ratings.sort_values(["userId","timestamp"])
last_idx = r_sorted.groupby("userId").tail(1).index #AGRUPANDO por usuario
test_df = r_sorted.loc[last_idx]  #se usa la última observación de cada usuario
train_df = r_sorted.drop(index=last_idx)

# Popularidad de items (misma referencia para ambos modelos) - conteo de las interacciones
pop = train_df["movieId"].value_counts().to_dict()

#  BASELINE SVD

reader = Reader(rating_scale=(0.5, 5.0))
data_surprise = Dataset.load_from_df(train_df[["userId","movieId","rating"]], reader)
trainset = data_surprise.build_full_trainset() # estructura de datos interna y optimizada para algoritmos de surprise

algo = SVD(n_factors=50, n_epochs=20, random_state=42)
algo.fit(trainset) #Aquí se lleva a cabo la factorización

# Predicciones y RMSE
# recolecta las calificaciones reales y las calificaciones predichas por el modelo SVD 
# para cada una de las interacciones en el conjunto de prueba:
y_true_svd, y_pred_svd = [], []
for r in test_df.itertuples(index=False):
    y_true_svd.append(float(r.rating))
    y_pred_svd.append(algo.predict(r.userId, r.movieId).est) #calificación estimada del modelo

rmse_svd = mean_squared_error(y_true_svd, y_pred_svd) # !!!!!!!!!!!!!!!!!! #, squared=False
print("RMSE Baseline (SVD sin IPS):", rmse_svd)

# Función de recomendaciones con SVD
def recs_svd(uid, n=10):
    all_items = ratings["movieId"].unique()
    seen = set(train_df[train_df["userId"] == uid]["movieId"]) #si ya las vio el usuario
    preds = [(iid, algo.predict(uid, iid).est) for iid in all_items if iid not in seen] #predice sobre lo no visto
    preds.sort(key=lambda x: -x[1]) #ordena de mayor a menor calif predicha las no vistas
    return [iid for iid, _ in preds[:n]] #devuelve los IDs de las primeras n peículas de esa lista ordenada

#  LIGHTFM con IPS 

# Reindexado usuarios/items para que queden de la forma 0...N-1
uids = {u:i for i,u in enumerate(sorted(ratings["userId"].unique()))} #u es el id del usuario
iids = {m:i for i,m in enumerate(sorted(ratings["movieId"].unique()))} #m es el id de la película
U, I = len(uids), len(iids)

# Matriz de interacciones train
R = sparse.lil_matrix((U, I), dtype=np.float32) #declaro la matrix
for r in train_df.itertuples(index=False):
    R[uids[r.userId], iids[r.movieId]] = float(r.rating) #llenar la matriz

#  PROPENSIÓN (modelo logístico)

# Construimos dataset de exposición (positivos = interacciones en train, negativos = muestreados)
ua = train_df["userId"].value_counts().to_dict()
ip = train_df["movieId"].value_counts().to_dict()

rows = []
# positivos
for r in train_df.itertuples(index=False):
    rows.append([ua[r.userId], ip[r.movieId], 1])
# negativos (pares no observados)
users = ratings["userId"].unique()
items = ratings["movieId"].unique()
for _ in range(len(train_df)):
    u = random.choice(users) #ojo aquí
    i = random.choice(items)
    if not ((train_df["userId"]==u) & (train_df["movieId"]==i)).any():
        rows.append([ua.get(u,0), ip.get(i,0), 0])

exp_df = pd.DataFrame(rows, columns=["ua","ip","exposed"])

# Entrenamos regresión logística para estimar propensity score
X = exp_df[["ua","ip"]]
y = exp_df["exposed"]
logit = LogisticRegression(max_iter=500)
logit.fit(X, y)

# Definimos e_ui a partir del modelo aprendido
def e_ui(u,i):
    return logit.predict_proba([[ua.get(u,0), ip.get(i,0)]])[0,1]

# Construimos matriz de pesos IPS
W = sparse.lil_matrix((U,I), dtype=np.float32) #definiendo la estructura de la sparse matrix
for r in train_df.itertuples(index=False):
    W[uids[r.userId], iids[r.movieId]] = 1/max(e_ui(r.userId, r.movieId), 1e-6) #llenando la matriz

#  Entrenamiento LightFM 

model = LightFM(no_components=32, loss='warp', random_state=42)
model.fit(R.tocsr(), sample_weight=W.tocoo(), epochs=15, num_threads=2)

# Predicciones y RMSE
y_true_lfm, y_pred_lfm = [], []
for r in test_df.itertuples(index=False):
    u_idx, i_idx = uids[r.userId], iids[r.movieId]
    y_true_lfm.append(float(r.rating))
    y_pred_lfm.append(model.predict(u_idx, np.array([i_idx]))[0])

rmse_lfm = mean_squared_error(y_true_lfm, y_pred_lfm) # !!!!!!!!!!!!!!!, squared=False
print("RMSE LightFM + IPS (logit prop):", rmse_lfm)

# Función de recomendaciones con LightFM
def recommend_lightfm(uid_raw, n=10):
    uid = uids[uid_raw]
    scores = model.predict(uid, np.arange(I))
    seen = set(train_df[train_df["userId"] == uid_raw]["movieId"])
    recs = [(mid, scores[iids[mid]]) for mid in iids if mid not in seen]
    recs.sort(key=lambda x: -x[1])
    return [mid for mid, _ in recs[:n]]

#  Popularidad media (Top-1000) 
# Mide si las recomendaciones de cada modelo se concentran en ítems populares o si logran diversificar.
def avg_pop(method_fn, users_eval, n=1000):
    vals = []
    for u in users_eval:
        recs = method_fn(u, n=n)
        vals.append(np.mean([pop.get(m, 0) for m in recs]))
    return np.mean(vals)

users_eval = list(test_df["userId"].unique())[:500]

print("Popularidad media Top-1000 (Baseline SVD):", avg_pop(recs_svd, users_eval, n=1000))
print("Popularidad media Top-1000 (LightFM IPS):", avg_pop(recommend_lightfm, users_eval, n=1000))


RMSE Baseline (SVD sin IPS): 0.9373953905032041
RMSE LightFM + IPS (logit prop): 33.4377282628473
Popularidad media Top-1000 (Baseline SVD): 31.251744
Popularidad media Top-1000 (LightFM IPS): 42.255228


In [10]:
def coverage(method_fn, users_eval, n=1000):
    rec_items = set()
    for u in users_eval:
        recs = method_fn(u, n=n)
        rec_items.update(recs)
    total_items = len(ratings["movieId"].unique())
    return len(rec_items) / total_items

print("Coverage Top-1000 (Baseline SVD):", coverage(recs_svd, users_eval, n=1000))
print("Coverage Top-1000 (LightFM IPS):", coverage(recommend_lightfm, users_eval, n=1000))


Coverage Top-1000 (Baseline SVD): 0.6826408885232415
Coverage Top-1000 (LightFM IPS): 0.7519539284245167


# Conclusiones
- Los **confusores** sesgan la exposición y refuerzan lo popular.
- El **baseline (SVD sin corrección)** recomienda ítems más populares.
- Con **IPS (LightFM con sample_weight)** equilibramos el entrenamiento:
  - Similar RMSE, pero recomendaciones menos sesgadas hacia la popularidad.
  - Mayor diversidad en el ranking.
- En producción: modelar propensión con más señales (posición, campaña, dispositivo).
